In [1]:
import pandas as pd

from libsAndPackages import *

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of BertModel were not initialized from the model checkpoint at undertheseanlp/vietnamese-ner-v1.4.0a2 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# isComparative binary classification

In [3]:
isComparativeData = pd.read_csv(curDir + "//datasets//modified//preprocessed//VLSP23_isComparative.csv", index_col = 'Unnamed: 0')
isComparativeData = isComparativeData.reset_index(drop = True)
isComparativeData

,Input sentence,isComparative
0,"Title: Review iPhone 14: Thiết kế, hiệu năng, ...",0
1,"iPhone 14 được ra mắt vào 09/2022, được đánh g...",1
2,Về ngoại hình thì cũng có rất ít sự khác biệt ...,1
3,"Có một vài ý kiến trái chiều cho rằng, phiên b...",0
4,Thực tế khi mua hoặc nâng cấp một thiết bị di ...,0
...,...,...
3432,Trong khi trên iPhone 14 Pro Max vẫn còn 21%.,0
3433,Kết quả cho thấy model của Apple có thể ...,0
3434,Đối thủ của iPhone 14 Pro Max - Pixel 7 Pro...,0
3435,"Tuy nhiên, một lưu ý rằng Google Pixel 7 Pr...",1


In [4]:
isComparativeData['isComparative'].value_counts()

0    2640
1     797
Name: isComparative, dtype: int64

In [5]:
xDataLowercase = np.array([text.lower() for text in isComparativeData['Input sentence']])
xDataLowercaseSegmented = np.array([rdrSegmenter.word_segment(text) for text in xDataLowercase])
xDataSegmented = np.array([rdrSegmenter.word_segment(text) for text in isComparativeData['Input sentence']])

yData = isComparativeData['isComparative']

C:\Users\nguye\AppData\Local\Temp\ipykernel_21892\3315150353.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  xDataLowercaseSegmented = np.array([rdrSegmenter.word_segment(text) for text in xDataLowercase])
C:\Users\nguye\AppData\Local\Temp\ipykernel_21892\3315150353.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  xDataSegmented = np.array([rdrSegmenter.word_segment(text) for text in isComparativeData['Input sentence']])


In [6]:
yDataForSegmented = []
for i in range(xDataSegmented.shape[0]):
    yDataForSegmented.append([yData[i] for _ in range(len(xDataSegmented[i]))])
yDataForTokenize = np.array(yDataForSegmented)

C:\Users\nguye\AppData\Local\Temp\ipykernel_21892\2073613290.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  yDataForTokenize = np.array(yDataForSegmented)


In [7]:
xDataLowercaseSegmented = np.concatenate(xDataLowercaseSegmented)
xDataSegmented = np.concatenate(xDataSegmented)
yDataForSegmented = np.concatenate((yDataForSegmented))

In [8]:
tfidfVectorizer = TfidfVectorizer(max_features = 3000, sublinear_tf = True)
xDataSegmentedVectorized = tfidfVectorizer.fit_transform(xDataSegmented)
xDataSegmentedVectorized

<3485x3000 sparse matrix of type '<class 'numpy.float64'>'
	with 51459 stored elements in Compressed Sparse Row format>

In [9]:
xDataSegmentedTokenized = phobertTokenizer(text = list(xDataSegmented), is_split_into_words = False, padding = True)

xDataSegmentedPhoBERTFeature = []
with torch.no_grad():
    for i in range(len(xDataSegmentedTokenized.input_ids)):
        xDataSegmentedPhoBERTFeature.append(phobertFeatureExtractor(
            input_ids = torch.unsqueeze(torch.tensor(xDataSegmentedTokenized['input_ids'][i]), 0),
            token_type_ids = torch.unsqueeze(torch.tensor(xDataSegmentedTokenized['token_type_ids'][i]), 0),
            attention_mask = torch.unsqueeze(torch.tensor(xDataSegmentedTokenized['attention_mask'][i]), 0)
        )['pooler_output'])

In [10]:
xDataSegmentedPhoBERTFeatureTensor = torch.concat(xDataSegmentedPhoBERTFeature)

In [12]:
# pd.DataFrame(xDataSegmentedPhoBERTFeatureTensor.numpy()).to_csv(r"C:\Users\nguye\Downloads\VLSP23_xDataSegmentedPhoBERTFeature.csv")

## Split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(xDataSegmentedPhoBERTFeatureTensor, yDataForSegmented, test_size = 0.2, random_state = 10, shuffle = True)
print("XTrain shape = %s, yTrain shape = %s" % (X_train.shape, y_train.shape))
print("XTest shape = %s, yTest shape = %s" % (X_test.shape, y_test.shape))

XTrain shape = torch.Size([2788, 768]), yTrain shape = (2788,)
XTest shape = torch.Size([697, 768]), yTest shape = (697,)


In [14]:
pd.Series(y_train).value_counts()

0    2138
1     650
dtype: int64

## Modeling

### LogReg

In [15]:
modelLogReg = LogisticRegression()
modelLogReg.fit(X_train, y_train)

print("On training set:\n%s" % (classification_report(y_train, modelLogReg.predict(X_train))))

y_pre = modelLogReg.predict(X_test)
print("On test set:\n%s" % (classification_report(y_test, y_pre)))

On training set:
              precision    recall  f1-score   support

           0       0.91      0.96      0.94      2138
           1       0.85      0.70      0.77       650

    accuracy                           0.90      2788
   macro avg       0.88      0.83      0.85      2788
weighted avg       0.90      0.90      0.90      2788

On test set:
              precision    recall  f1-score   support

           0       0.88      0.95      0.91       532
           1       0.78      0.58      0.67       165

    accuracy                           0.86       697
   macro avg       0.83      0.77      0.79       697
weighted avg       0.86      0.86      0.85       697


C:\Users\nguye\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### SVM

In [16]:
modelSVM = SVC()
modelSVM.fit(X_train, y_train)

print("On training set:\n%s" % (classification_report(y_train, modelSVM.predict(X_train))))

y_pre = modelSVM.predict(X_test)
print("On test set:\n%s" % (classification_report(y_test, y_pre)))

On training set:
              precision    recall  f1-score   support

           0       0.86      0.98      0.92      2138
           1       0.88      0.49      0.63       650

    accuracy                           0.87      2788
   macro avg       0.87      0.73      0.77      2788
weighted avg       0.87      0.87      0.85      2788
On test set:
              precision    recall  f1-score   support

           0       0.81      0.97      0.88       532
           1       0.74      0.29      0.42       165

    accuracy                           0.81       697
   macro avg       0.78      0.63      0.65       697
weighted avg       0.80      0.81      0.77       697
